## CIFAR 10

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from IPython.core.display import display
from PIL import Image

In [3]:
from fastai.conv_learner import *
from fastai.models.cifar10.wideresnet import wrn_22
torch.backends.cudnn.benchmark = True
PATH = Path("data/cifar10/")
os.makedirs(PATH,exist_ok=True)

In [4]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

bs=512*3#(512*4*1.3)
sz=32

In [5]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)

In [6]:
m = wrn_22()
m = nn.DataParallel(m, [0,1,3])

In [9]:
learn = ConvLearner.from_model_data(m, data)

In [10]:
learn.crit = nn.CrossEntropyLoss()
learn.metrics = [accuracy]

In [ ]:
for x in range(1,4):
    wd= (5-x)*1e-4
    print('\nwd: ' + str(wd))
    #learn.lr_find2(wds=wd, start_lr=0.01, end_lr=10, num_it=100)
    learn.lr_find(wds=wd, start_lr=0.1, end_lr=100)
    learn.sched.plot(n_skip=0, n_skip_end=0)
    display(Image.open('lr_loss_plot.png'))

In [11]:
wd=2e-4
lr=1.7

In [12]:
%time learn.fit(lr, 1, wds=wd, cycle_len=30, use_clr_beta=(20,20,0.95,0.85))

epoch      trn_loss   val_loss   accuracy                 
    0      1.836099   2.906796   0.139     
    1      1.554884   2.171634   0.3539                   
    2      1.320663   1.394464   0.544                    
    3      1.124831   1.302039   0.565                    
    4      0.977513   1.215186   0.6012                    
    5      0.839722   0.968528   0.6671                    
    6      0.730589   0.69855    0.7533                    
    7      0.646078   0.737501   0.7444                    
    8      0.577169   0.980248   0.6845                    
    9      0.52127    0.741868   0.7463                    
    10     0.469514   0.742958   0.7524                    
    11     0.433083   0.902592   0.711                     
    12     0.394949   0.705105   0.7791                    
    13     0.369589   0.690192   0.7604                    
    14     0.350849   0.655356   0.783                     
    15     0.321433   0.752563   0.7647                    


[array([0.24062]), 0.9289000024795532]